In [2]:
from parse_deck import parse_deck

In [ ]:
# parse from blog

decks = []

# decks = [
#     deck category 1: [
#         {
#             deck_code: "",
#         }, ...
#     ],
#     deck category 2:...
# ]

In [ ]:
# update deck info
    
# decks = [
#     deck category 1: [
#         {
#             deck_code: "",
#             pekomons: {},
#             tools: {},
#             supporters: {},
#             stages: {},
#             energies: {},
#             rank: 1,
#             no_people: 32,
#             date: datetime
#         }, ...
#     ],
#     deck category 2:...
# ]